In [2]:
import matplotlib
%matplotlib tk
#matplotlib.use('Agg')

%load_ext autoreload
%autoreload 2

%autosave 180

import matplotlib.pyplot as plt
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

import numpy as np
import os

import sys
sys.path.append('..')
from visualize import visualize_svm as Visualize

from tqdm import trange, tqdm

sys.path.append('../..')
from svm import svm as SVM

import scipy
from statsmodels.stats.multitest import multipletests

#
import pandas as pd
import matplotlib.patches as mpatches
import scipy.stats as stats


Autosaving every 180 seconds


<ipython-input-2-2ad95b854a16>:11: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import display, HTML


In [3]:
# #######################################################
# #### RERUN SVM BUT LOWPASS FILTER LOCANMF COMPS #######
# #######################################################

#
def load_supersessions(animal_id, lockout):
    fname = os.path.join('/media/cat/4TBSSD/yuki/',
                         animal_id,
                         'super_sessions',
                         'alldata_body_and_nonreward_lockout_'+str(lockout)+'secLockout_[]bodyfeats.npz')

    data = np.load(fname, allow_pickle=True)
    trials = np.vstack(data['trials'])
    random = np.vstack(data['random'])
    
    return trials, random

# 
# def filter_trace(trace, smooth_window=30):
#     box = np.ones(smooth_window)/smooth_window
#     trace_smooth = np.convolve(trace, box, mode='valid')
#     return trace_smooth

from scipy.signal import butter, lfilter, filtfilt, hilbert, chirp

#
def butter_lowpass(cutoff, nyq_freq, order=4):
    normal_cutoff = float(cutoff) / nyq_freq
    b, a = butter(order, normal_cutoff, btype='lowpass')
    return b, a

#
def butter_lowpass_filter(data, cutoff_freq, nyq_freq, order=4):
    # Source: https://github.com/guillaume-chevalier/filtering-stft-and-laplace-transform
    b, a = butter_lowpass(cutoff_freq, nyq_freq, order=order)
    y = filtfilt(b, a, data)
    return y

#
def compute_significance(data):

    #
    significance = 0.05

    #
    sig = []
    for k in range(data.shape[0]):

        #
        res = scipy.stats.ttest_1samp(data[k], 0.5)
        sig.append(res[1])

    # multiple hypothesis test Benjamini-Hockberg
    temp = np.array(sig)
    temp2 = multipletests(temp, 
                          alpha=significance, 
                          method='fdr_bh')
    sig = temp2[1]

    # find earliest significant;
    #print ("sig (BH corrected):", sig)
    edt = sig.shape[0]
    for k in range(sig.shape[0]-1,0,-1):
        if sig[k]<=significance:
            edt = k+1
        else:
            break

    # 
    edtsec = -(sig.shape[0]-edt)/30.

    # 
    return edtsec

#
def plot_edt_distributions_box_plots(window,
                                     animal_id, 
                                     clrs_local, 
                                     lockout,
                                     filtered_flag):

    #
    codes = ['all', 'all-subsampled','lockedout']

    #
    vis = Visualize.Visualize()
    
    legend_size = 22
        
    # 
    shift= 15
    linestyles = ['-','-','-','-','-','-','-','-']

    #titles = ['all','lockout','concatenated']
    pvals = [0.05,0.01,0.001,0.0001,0.00001]
    edts = []
    yvals = []

    # 
    ########## LOAD NON FILTERED FILE ##########
    # for some reason the manually recomputed EDT above gives some sprious results for IA2 (i.e. much too low)
    #   so just load the original computed edts from initial submission
    if False:
        fname1 = os.path.join('/media/cat/4TBSSD/yuki/'+animal_id+'_edts_locked_out_'+str(lockout)+'.npz')
        res1 = np.load(fname1, allow_pickle=True)['all_res_continuous']

        # remove artficat results below -15 sec
        # if False:
        #     idx = np.where(res1<-15)[0]
        #     res1 = np.delete(res1, idx)
        #     if idx.shape[0]>0:
        #         print ("errros found: ", idx.shape)
        
    else:
        # or load old data:
            
        fname1 = os.path.join('/media/cat/4TBSSD/yuki/'+animal_id+'_edts_lockedout.npy')
        res1 = np.array(np.load(fname1,allow_pickle=True))
        times = res1[:,0]
        res2 = res1[:,1]
        idx = np.where(times==lockout)[0]
        res1 = np.hstack(res2[idx])
        #print (res1)
        
        # subselect every 4th sessoin to process only non-overlapping hybrid sessions
        if window==200:
            res1 = res1[::4]

            
    edts.append(np.hstack(res1))
    yvals.append(np.arange(len(res1)))
   # print (animal_id, "average EDT filtered: ", np.mean(edts))

    
        
    fname2 = os.path.join('/media/cat/4TBSSD/yuki/'+animal_id+'_edts_locked_out_filtered_'+str(lockout)+'.npz')
    
    
    ########## LOAD 2ND FILE ##########
    res2 = np.load(fname2, allow_pickle=True)['all_res_continuous']
            
    # 
    edts.append(np.hstack(res2))
    yvals.append(np.arange(len(res2)))

    #print ("edts: ", edts)
    print (animal_id, "average EDT filtered: ", np.mean(res2))
    
    # print differences between means
    #print (animal_id, "  diff in means: ", np.mean(res1) - np.mean(res2))
    
    # 
    my_dict = dict(Lockout = edts[0], 
                   Lockout_filter = edts[1],
                 )
    
    # 
    data = pd.DataFrame.from_dict(my_dict, orient='index')
    data = data.transpose()

    ######################################################### 
    ################## SCATTER PLOTS ########################
    ######################################################### 
    for i,d in enumerate(data):
        y = data[d]
        x = np.random.normal(i+1, 0.04, len(y))
        plt.plot(x, y, 
                 mfc =clrs_local[i], 
                 mec='k', 
                 ms=10, 
                 marker="o", 
                 linestyle="None",
                 alpha=.8

                )


    patches = []
    for c,code in enumerate(codes):
        patches.append(mpatches.Patch(color=clrs_local[c], label=code))

    ######################################################### 
    ###################### BOX PLOTS ########################
    ######################################################### 
    flierprops = dict(marker='o', 
                      #markerfacecolor='g', 
                      #markersize=10000,
                      linestyle='none', 
                      markeredgecolor='r')

    #
    data.boxplot(showfliers=False,
                flierprops=flierprops)

    temp = np.hstack(yvals)
    ymax = np.max(temp)

    #
    plt.xlim(0.5, 4)

    #
    plt.plot([0,6.5], [-3,-3],'--',linewidth=3,c='grey',alpha=.5)
    plt.plot([0,6.5],[-10,-10],'--',linewidth=3,c='grey',alpha=.5)
    plt.plot([0,6.5],[-5,-5],'--',linewidth=3,c='grey',alpha=.5)

    # ax2 = ax.twinx()  # instantiate a second axes that shares the same x-axis

    patches = []
    for p in [1]:
        res = stats.ks_2samp(edts[0], edts[p])
        #print ("res: ", res)
        label_ = ''
        for k in range(len(pvals)):
            if res[1]<pvals[k]:
                label_ = label_ + "*"
            else:
                break

        patches.append(mpatches.Patch(color=clrs_local[p-1],label=label_))

    plt.xticks(rotation=45)
    #
    plt.legend(handles=patches,
               fontsize=15)
    plt.ylim(-15,0)


In [4]:
#######################################################
#### LOAD SUPER SESSIONS AND LOWPASS FILTER THEM ######
#######################################################
#
lockouts = [0,6,9,12,15,18,21]

#
for lockout in lockouts:
    #lockout=15
    cutoff_freq = 0.3
    nyq_freq = 30

    #
    animal_ids = ['IA1','IA2','IA3','IJ1','IJ2','AQ2']

    # 
    for animal_id in animal_ids:
        fname_out = os.path.join('/media/cat/4TBSSD/yuki/',
                             animal_id,
                             'super_sessions',
                             'alldata_body_and_nonreward_lockout_'+str(lockout)+'secLockout_[]bodyfeats_0.3hz.npz')
        #
        if os.path.exists(fname_out):
            continue
        
        print (animal_id, lockout)

        #
        trials, random = load_supersessions(animal_id, lockout)
        print (trials.shape)

        #
        for k in trange(trials.shape[0]):
            for p in range(trials.shape[2]):
                trials[k,:,p] = butter_lowpass_filter(trials[k,:,p],cutoff_freq, nyq_freq)
                random[k,:,p] = butter_lowpass_filter(random[k,:,p],cutoff_freq, nyq_freq)

        np.savez(fname_out,
                 trials = trials,
                 random = random)
print ("DONE")

DONE


In [5]:
#######################################################
########### RUN SVM ON FILTERED SUPER SESSIONS ########
#######################################################
#animal_ids = ['IA1','IA2','IA3','IJ1','IJ2','AQ2']
animal_ids = ['AQ2'] #,'AQ2']

#
#lockouts = [15]
lockouts = [0,6,9,12,15,18,21]

for lockout in lockouts[::-1]:
    for animal_id in animal_ids:
        # animal_id = 'AQ2'

        print ("animal id: ", animal_id)
        fname = os.path.join('/media/cat/4TBSSD/yuki/',
                             animal_id,
                             'super_sessions',
                             'alldata_body_and_nonreward_lockout_'+str(lockout)+'secLockout_[]bodyfeats_0.3hz.npz')

        # 
        data = np.load(fname, allow_pickle=True)
        trials = data['trials'].transpose(0,2,1)
        random = data['random'].transpose(0,2,1)
        print ("trials: ", trials.shape)

        # 
        svm = SVM.PredictSVMChoiceSuperSession()
        svm.xvalidation = 10
        svm.parallel = True

        # window parameters
        svm.window = 30              # seconds of prediction window, backwards in time in seconds; e.g. -30sec ... +30sec
        svm.lockout_window = 10      # time for locking out other pulls 
        svm.sliding_window = 30      # number of frames in sliding window

        # pca params
        svm.pca_flag = True
        svm.pca_var = 0.95        # this is default for now for nComp = 20

        # svm parameters
        svm.xvalidation = 10      # KFold xvalidation step
        svm.data_split = 0.8      # split 80/20 data for training
        svm.method = 'sigmoid'    # method used for computing SVM

        # run-time parameters
        svm.parallel = True
        svm.n_cores = svm.xvalidation
        svm.min_trials = 30
        svm.overwrite = False
        svm.gpu_flag = False

        #
        fname_out = fname

        # this function is the latest version for filtered data
        # it saves the output to fname_save = fname_out.replace('.npz','_'+str(k)+"_super_res.npz")
        svm.predict3(trials, random, fname_out)



animal id:  AQ2
trials:  (318, 16, 1800)
   start:  0  end:  200  /  318
   start:  50  end:  250  /  318
   start:  100  end:  300  /  318
   start:  150  end:  350  /  318
   start:  200  end:  400  /  318
   start:  250  end:  450  /  318
animal id:  AQ2
trials:  (412, 16, 1800)
   start:  0  end:  200  /  412
   start:  50  end:  250  /  412
   start:  100  end:  300  /  412
   start:  150  end:  350  /  412
   start:  200  end:  400  /  412
   start:  250  end:  450  /  412
   start:  300  end:  500  /  412
   start:  350  end:  550  /  412
animal id:  AQ2
trials:  (594, 16, 1800)
animal id:  AQ2
trials:  (910, 16, 1800)
animal id:  AQ2
trials:  (1419, 16, 1800)
   start:  0  end:  200  /  1419
   start:  50  end:  250  /  1419
   start:  100  end:  300  /  1419
   start:  150  end:  350  /  1419
   start:  200  end:  400  /  1419
   start:  250  end:  450  /  1419
   start:  300  end:  500  /  1419
   start:  350  end:  550  /  1419
   start:  400  end:  600  /  1419
   start:  4

In [35]:
###############################################
###### MAKE FIRST DECODING TIMES FILES ########
###############################################
import scipy
from statsmodels.stats.multitest import multipletests

root_dir = '/media/cat/4TBSSD/yuki'

#
animal_ids = ['IA1','IA2','IA3','IJ1','IJ2','AQ2']
#animal_ids = ['IA2']
lockout = 15
window = 50
for animal_id in animal_ids:

    # load
    ctr=0
    edts = []
    for n in range(0, 10000, window):

        fname = os.path.join(root_dir,
                             animal_id,
                             'super_sessions',
                            # 'alldata_body_and_nonreward_lockout_0secLockout_[]bodyfeats_super_res_subsample_'
                             'alldata_body_and_nonreward_lockout_'+str(lockout)+'secLockout_[]bodyfeats_0.3hz_'+
                             str(n)+'_super_res.npz')

        try:
            data = np.load(fname,
                      allow_pickle=True)
        except:
            print ("Done ...  ",animal_id)
            break

        acc = data['accuracy']
        #print ("acc: ", acc.shape)

        # 
        #acc2 = []
        for k in range(acc.shape[1]):
            acc[:,k] = butter_lowpass_filter(acc[:,k],0.5, 30)

        # 
        edt = compute_significance(acc[:900])
        
        # must add 1 sec because of SVM works in 1 sec chunks
        edt = edt+1
        if edt>0:
            edt=0
            
        #
        edts.append(edt)
        ctr+=1
    print ("total #: ", ctr)

    #
    np.savez('/media/cat/4TBSSD/yuki/'+animal_id+"_edts_locked_out_filtered_"+str(lockout)+".npz", 
             all_res_continuous = edts)
    
    #
    
    
    #
    print ("")
    print ("")
    
    
#
print ("DONE ALL") 

Done ...   IA1
total #:  10


Done ...   IA2
total #:  11


Done ...   IA3
total #:  4


Done ...   IJ1
total #:  5


Done ...   IJ2
total #:  3


Done ...   AQ2
total #:  12


DONE ALL


In [28]:
###############################################################
###############################################################
###############################################################
    
#
lockout = 15
window = 50
#############################################
animal_ids = ['IA1','IA2','IA3','IJ1','IJ2','AQ2']
#animal_ids = ['IA1']
lockout=15
clrs_local = ['red','green','yellow']
filtered_flag = True

#
ctr=0
fig=plt.figure(figsize=(20,5))
diffs = []
for animal_id in animal_ids:
    ax=plt.subplot(1,6,ctr+1)
    
    diff = plot_edt_distributions_box_plots(window, 
                                            animal_id,
                                            clrs_local,
                                            lockout,
                                            filtered_flag)
    ctr+=1
    
plt.suptitle("Lockout "+str(lockout))
#######################################
if True:
    plt.savefig('/home/cat/filtered_EDTs_nonoverlapping.svg')
    plt.close()
else:
    plt.show()

IA1 average EDT filtered:  -3.5
IA2 average EDT filtered:  -4.854545454545455
IA3 average EDT filtered:  -6.949999999999999
IJ1 average EDT filtered:  -4.306666666666667
IJ2 average EDT filtered:  -3.0
AQ2 average EDT filtered:  -3.6999999999999997


In [ ]:
                                                    /media/cat/4TBSSD/yuki/IA1_edts_locked_out.npz

In [18]:
# ############################################
# ############################################
# ############################################
fname = '/home/cat/IA1_edts_lockedout.npy'
data = np.load(fname, allow_pickle=True)
lockouts = data[:,0]
print (lockouts)


[0 6 9 12 15 18 21 24 27 30]
[0
 list([-2.8666666666666667, -3.0, -3.466666666666667, -3.1, -7.566666666666666, -7.533333333333333, -3.533333333333333, -3.533333333333333, -3.2333333333333334, -3.3333333333333335, -3.066666666666667, -2.933333333333333, -3.4, -2.8, -3.0, -2.7333333333333334, -2.566666666666667, -2.3, -1.9, -1.5666666666666667, -1.6666666666666667, -1.7666666666666666, -1.3666666666666667, -1.4, -1.3, -0.7666666666666667, -1.4])]


In [156]:
# #############################################
# #############################################
# #############################################
# session_name = 'IA1pm_Feb23_30Hz'

# trials = np.load('/media/cat/4TBSSD/yuki/IA1/tif_files/'+session_name+'/'+session_name + '_code_04_trial_ROItimeCourses_30sec_pca_0.95_.3hzfiltered.npy')
# trials2 = np.load('/media/cat/4TBSSD/yuki/IA1/tif_files/'+session_name+'/'+session_name+'_code_04_trial_ROItimeCourses_30sec_pca_0.95.npy')
# random = np.load('/media/cat/4TBSSD/yuki/IA1/tif_files/'+session_name+'/'+session_name + '_code_04_random_ROItimeCourses_30sec_pca_0.95_.3hzfiltered.npy')
# random2 = np.load('/media/cat/4TBSSD/yuki/IA1/tif_files/'+session_name+'/'+session_name + '_code_04_random_ROItimeCourses_30sec_pca_0.95.npy')

# print (trials.shape)
# print (random.shape)

# #
# t = np.arange(trials.shape[2])/30.-30

# #
# plt.figure(figsize=(20,10))
# ax1=plt.subplot(2,1,1)
# plt.plot(t,trials2[0,0].T,c='blue',alpha=1,label='unfiltered')
# plt.plot(t,trials[0,0].T,c='green',alpha=1,label='filtered')
# plt.legend(fontsize=20)
# plt.plot([-30,30], [0,0],'--',linewidth=2,c='grey',alpha=.5)
# plt.plot([0,0],[-10,10],'--',linewidth=2,c='grey',alpha=.5)
# plt.xlabel("Time (sec)")
# plt.ylabel("Trial")
# plt.xlim(-30,30)
# plt.ylim(-10,10)    
# #
# ax1=plt.subplot(2,1,2)
# plt.plot(t,random2[0,0].T,c='red',alpha=1,label='unfiltered')
# plt.plot(t,random[0,0].T,c='green',alpha=1,label='filtered')
# plt.legend(fontsize=20)
# plt.plot([-30,30], [0,0],'--',linewidth=2,c='grey',alpha=.5)
# plt.plot([0,0],[-10,10],'--',linewidth=2,c='grey',alpha=.5)
# plt.xlabel("Time (sec)")
# plt.ylabel("Random")

# if False:
#     plt.savefig('/home/cat/trials.svg')
#     plt.close()
# plt.show()


(81, 8, 1801)
(76, 8, 1801)


In [51]:
############################################
########## QUICK AND DIRTY CHECKS ##########
############################################
#session_name = 'IA1pm_Feb23_30Hz'
fnames = [

    # '/media/cat/4TBSSD/yuki/IA3/super_sessions/alldata_body_and_nonreward_lockout_15secLockout_[]bodyfeats_0.3hz_400_super_res.npz',
    # '/media/cat/4TBSSD/yuki/IA3/super_sessions/alldata_body_and_nonreward_lockout_15secLockout_[]bodyfeats_0.3hz_350_super_res.npz',
    # '/media/cat/4TBSSD/yuki/IA3/super_sessions/alldata_body_and_nonreward_lockout_15secLockout_[]bodyfeats_0.3hz_300_super_res.npz',
    # '/media/cat/4TBSSD/yuki/IA3/super_sessions/alldata_body_and_nonreward_lockout_15secLockout_[]bodyfeats_0.3hz_250_super_res.npz',
    # '/media/cat/4TBSSD/yuki/IA3/super_sessions/alldata_body_and_nonreward_lockout_15secLockout_[]bodyfeats_0.3hz_200_super_res.npz',
    # '/media/cat/4TBSSD/yuki/IA3/super_sessions/alldata_body_and_nonreward_lockout_15secLockout_[]bodyfeats_0.3hz_150_super_res.npz',
    # '/media/cat/4TBSSD/yuki/IA3/super_sessions/alldata_body_and_nonreward_lockout_15secLockout_[]bodyfeats_0.3hz_100_super_res.npz',
    # '/media/cat/4TBSSD/yuki/IA3/super_sessions/alldata_body_and_nonreward_lockout_15secLockout_[]bodyfeats_0.3hz_50_super_res.npz',
    # '/media/cat/4TBSSD/yuki/IA3/super_sessions/alldata_body_and_nonreward_lockout_15secLockout_[]bodyfeats_0.3hz_0_super_res.npz',

    #'/media/cat/4TBSSD/yuki/IA1/super_sessions/alldata_body_and_nonreward_lockout_15secLockout_[]bodyfeats_0_super_res.npz',
    #'/media/cat/4TBSSD/yuki/IA1/super_sessions/alldata_body_and_nonreward_lockout_15secLockout_[]bodyfeats_200_super_res.npz'

    #'/media/cat/4TBSSD/yuki/IA1/super_sessions/alldata_body_and_nonreward_lockout_15secLockout_[]bodyfeats_0.3hz_0_super_res.npz',
    #'/media/cat/4TBSSD/yuki/IA1/super_sessions/alldata_body_and_nonreward_lockout_15secLockout_[]bodyfeats_0.3hz_200_super_res.npz'
    '/media/cat/4TBSSD/yuki/IA2/super_sessions/alldata_body_and_nonreward_lockout_15secLockout_[]bodyfeats_200_super_res.npz',
    '/media/cat/4TBSSD/yuki/IA2/super_sessions/alldata_body_and_nonreward_lockout_15secLockout_[]bodyfeats_0.3hz_200_super_res.npz'
]

# 
fig=plt.figure()
ax=plt.subplot(111)
for ctr,fname in enumerate(fnames):
    
    data = np.load(fname)
    
    
    predictions = data['predictions']
    print ("n_trials: ", predictions.shape[1])
    acc = data['accuracy']
    #length = 1801-acc.shape[0]
    #acc = np.vstack(
    #                 (np.zeros((length,10), dtype=np.float32), 
    #                  acc))
    print (acc.shape)
    t = np.arange(acc.shape[0])/30-29
    mean = acc.mean(1)
    std = np.std(acc,1)
    plt.plot(t,mean, label=fname)
    ax.fill_between(t, mean+std,
                    mean-std,
                    alpha=.2)

plt.plot([-30,30],[0.5,0.5],'--',c='grey')
plt.ylim(0,1)
plt.xlim(-20,0)
#plt.legend(fontsize=6)
if False:
    plt.savefig('/home/cat/filtered_IA3.svg')
    plt.close()
else:
    plt.show()


####
# plt.figure()
# d1 = np.load(fname)
# acc1 = d1['accuracy']
# print (acc1.shape)

# plt.plot(acc1.mean(1), label='all freq')

# d2 = np.load(fname_filtered)
# acc2 = d2['accuracy']
# print (acc2.shape)

# plt.plot(acc2.mean(1), label='<0.3hz')
# plt.legend()
# plt.show()

n_trials:  400
(900, 10)
n_trials:  400
(1770, 10)


In [42]:
fname = '/media/cat/4TBSSD/yuki/IA1/first_decoding_time_0.3hzFilter.npz'
data=np.load(fname)

sessions = data['all_session_names']
print (sessions.shape)

(53,)
